# Lab 15 — Transfer Learning & Modern AI

> **강의 시간:** 약 2시간  
> **주제:** 사전학습 모델 재사용 — Feature Extraction, Fine-tuning, 데이터 효율성

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | 사전학습 ResNet18 구조 탐색 + Feature Extraction | 30분 |
| 2 | Fine-tuning 전략 — 차등 LR, Unfreeze 비교 | 30분 |
| 3 | 데이터 효율성 분석 + 오류 분석 | 30분 |
| 4 | Exercise | 30분 |

---

**데이터셋:** PIL 합성 도형 이미지 (인터넷 불필요, 코드 내 자동 생성)  
- 3 클래스: `circle`, `square`, `triangle`  
- Train 300/class (900개) · Test 100/class (300개), 96×96 PIL → Resize(224)  

> ⚠️ **사전학습 가중치 다운로드:** 최초 실행 시 ResNet18 가중치(~45MB)가 자동 다운로드됩니다.  
> GPU/MPS 사용 권장 — CPU에서 fine-tuning은 10~15분 소요될 수 있습니다.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import copy, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from PIL import Image, ImageDraw

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.models as models
import torchvision.transforms as transforms

_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'PyTorch     : {torch.__version__}')
print(f'Using device: {device}')

# ImageNet 정규화 상수 (pretrained ResNet 필수)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]


In [ ]:
# ── 합성 도형 이미지 데이터셋 (PIL 생성, 인터넷 불필요) ──────────────────────────

CLASS_NAMES  = ['circle', 'square', 'triangle']
N_TRAIN_PER  = 300   # 클래스당 학습 샘플 수
N_TEST_PER   = 100   # 클래스당 테스트 샘플 수

base_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

aug_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


def make_shape_image(label, size=96, seed=None):
    """PIL 도형 이미지 생성 (label: 0=circle, 1=square, 2=triangle)"""
    rng = np.random.RandomState(seed)
    bg  = tuple(int(v) for v in rng.randint(190, 241, 3))
    img = Image.new('RGB', (size, size), bg)
    drw = ImageDraw.Draw(img)
    col = tuple(int(v) for v in rng.randint(30, 151, 3))
    lo  = size // 4 + size // 5
    hi  = size - lo
    cx  = int(rng.randint(lo, max(lo + 1, hi)))
    cy  = int(rng.randint(lo, max(lo + 1, hi)))
    r   = int(rng.randint(size // 5, size // 3 + 1))
    if label == 0:     # Circle
        drw.ellipse([cx-r, cy-r, cx+r, cy+r], fill=col)
    elif label == 1:   # Square
        drw.rectangle([cx-r, cy-r, cx+r, cy+r], fill=col)
    else:              # Triangle
        drw.polygon([(cx, cy-r), (cx+r, cy+r), (cx-r, cy+r)], fill=col)
    return img


class ShapeDataset(Dataset):
    """PIL 도형 이미지 Dataset — 메모리에 PIL 이미지 저장 후 transform 적용"""
    def __init__(self, n_per_class, split='train', transform=None, seed=42):
        self.transform = transform if transform is not None else base_transform
        self.imgs, self.labels = [], []
        offset = 0 if split == 'train' else 10000  # train/test seed 분리
        for cls in range(len(CLASS_NAMES)):
            for i in range(n_per_class):
                self.imgs.append(make_shape_image(cls, size=96, seed=seed + cls * 1000 + offset + i))
                self.labels.append(cls)

    def __len__(self):  return len(self.imgs)

    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]


print('데이터셋 생성 중...')
t0 = time.time()
train_ds  = ShapeDataset(N_TRAIN_PER, split='train', transform=base_transform)
test_ds   = ShapeDataset(N_TEST_PER,  split='test',  transform=base_transform)
print(f'완료: {time.time()-t0:.1f}s  |  Train: {len(train_ds)}  Test: {len(test_ds)}')

train_ldr = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0, pin_memory=False)
test_ldr  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=0, pin_memory=False)

# ── 샘플 시각화 ──────────────────────────────────────────────────────────────
fig, axes = plt.subplots(3, 5, figsize=(12, 7))
for cls in range(3):
    for i in range(5):
        img = make_shape_image(cls, size=96, seed=42 + cls * 100 + i)
        axes[cls, i].imshow(img)
        axes[cls, i].axis('off')
        if i == 0:
            axes[cls, i].set_ylabel(CLASS_NAMES[cls], rotation=0, ha='right',
                                    labelpad=50, fontsize=11)
plt.suptitle('합성 도형 이미지 데이터셋 샘플 (3 클래스 × 5개)', fontsize=12)
plt.tight_layout(); plt.show()

# 정규화 후 텐서 확인
t, l = train_ds[0]
print(f'\n텐서 shape: {t.shape}  range: [{t.min():.2f}, {t.max():.2f}]')
print(f'클래스: {CLASS_NAMES[l]}')


---
## Part 1. 사전학습 모델 & Feature Extraction

### 1-1. 왜 Transfer Learning인가?

| 상황 | 추천 전략 |
|---|---|
| 데이터 적음 + 유사 도메인 | Feature Extraction (backbone 고정) |
| 데이터 보통 + 유사 도메인 | Fine-tuning (전체 또는 일부 unfreeze) |
| 데이터 많음 + 다른 도메인 | Fine-tuning (전체) 또는 scratch |
| 데이터 충분 + 동일 도메인 | 사전학습 모델 직접 사용 |

### 1-2. ResNet18 구조

```
입력 (3, 224, 224)
  │
  conv1 (64, 7×7, stride=2) → BN → ReLU → MaxPool  : (64, 56, 56)
  │
  layer1 : BasicBlock × 2  (64 → 64)                : (64, 56, 56)
  layer2 : BasicBlock × 2  (64 → 128, stride=2)     : (128, 28, 28)
  layer3 : BasicBlock × 2  (128 → 256, stride=2)    : (256, 14, 14)
  layer4 : BasicBlock × 2  (256 → 512, stride=2)    : (512, 7, 7)
  │
  AdaptiveAvgPool → (512, 1, 1) → Flatten → fc(512→1000)
```

**Transfer Learning에서:** fc(512→1000) → fc(512→n_classes) 교체

### 1-3. Feature Extraction 작동 원리

```python
for name, param in model.named_parameters():
    if 'fc' not in name:
        param.requires_grad = False   # backbone 동결
# → optimizer가 fc 파라미터만 업데이트
```

**장점:** 훈련 가능 파라미터 11.7M → **1,539개** (99.99% 감소!)


In [ ]:
# ── ResNet18 구조 탐색 + Conv1 필터 시각화 ────────────────────────────────────

# 사전학습 모델 로드 (최초 실행 시 ~45MB 다운로드)
try:
    _model_inspect = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
except AttributeError:  # torchvision < 0.13
    _model_inspect = models.resnet18(pretrained=True)

print('=== ResNet18 레이어별 파라미터 수 ===')
total_params = 0
layer_info = [
    ('conv1 + bn1',  ['conv1', 'bn1']),
    ('layer1',       ['layer1']),
    ('layer2',       ['layer2']),
    ('layer3',       ['layer3']),
    ('layer4',       ['layer4']),
    ('fc (head)',    ['fc']),
]
print(f'{"그룹":<18} {"파라미터 수":>12} {"누적":>12}')
print('-' * 46)
for group_name, prefixes in layer_info:
    n = sum(p.numel() for name, p in _model_inspect.named_parameters()
            if any(name.startswith(pf) for pf in prefixes))
    total_params += n
    print(f'{group_name:<18} {n:>12,} {total_params:>12,}')
print(f'{"총합":<18} {sum(p.numel() for p in _model_inspect.parameters()):>12,}')

# ── Conv1 필터 시각화 ──────────────────────────────────────────────────────
conv1_w = _model_inspect.conv1.weight.data.clone()   # (64, 3, 7, 7)

fig, axes = plt.subplots(4, 8, figsize=(14, 7))
for i, ax in enumerate(axes.flat):
    if i < 32:
        filt = conv1_w[i].permute(1, 2, 0).numpy()   # (7, 7, 3)
        filt = (filt - filt.min()) / (filt.max() - filt.min() + 1e-8)
        ax.imshow(filt)
    ax.axis('off')
plt.suptitle('ResNet18 Conv1 사전학습 필터 (64개 중 32개 표시, 7×7×3)\n'
             'ImageNet으로 학습된 에지·방향성·색상 특징 감지기', fontsize=11)
plt.tight_layout(); plt.show()

del _model_inspect   # 메모리 정리
print('\n💡 이 필터들이 도형 이미지의 에지와 경계를 감지하는 데 활용됩니다!')


In [ ]:
# ── 헬퍼 함수 및 Feature Extraction 실험 ────────────────────────────────────

def make_pretrained_resnet18(n_classes=3):
    """사전학습 ResNet18: fc만 n_classes로 교체"""
    try:
        m = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    except AttributeError:
        m = models.resnet18(pretrained=True)
    m.fc = nn.Linear(m.fc.in_features, n_classes)
    return m


def make_random_resnet18(n_classes=3):
    """랜덤 초기화 ResNet18: fc만 n_classes로 교체"""
    m = models.resnet18(weights=None)
    m.fc = nn.Linear(m.fc.in_features, n_classes)
    return m


def set_trainable(model, layer_names=None):
    """
    layer_names=None  → 전체 unfreeze (모두 학습)
    layer_names=[]    → 전체 freeze
    layer_names=['fc'] → 'fc'로 시작하는 파라미터만 학습
    """
    if layer_names is None:
        for p in model.parameters(): p.requires_grad = True
        return
    for name, param in model.named_parameters():
        param.requires_grad = any(name.startswith(ln) for ln in layer_names)


def count_params(model):
    return sum(p.numel() for p in model.parameters())


def count_trainable(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def train_model(model, train_loader, val_loader,
                n_epochs=20, lr=1e-3, optimizer_fn=None, verbose=True):
    """공용 학습 함수: CrossEntropyLoss + Adam, requires_grad 파라미터만 업데이트"""
    model = model.to(device)
    if optimizer_fn is None:
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = optim.Adam(params, lr=lr, weight_decay=1e-4)
    else:
        optimizer = optimizer_fn(model)
    criterion = nn.CrossEntropyLoss()
    train_losses, val_accs = [], []

    for epoch in range(n_epochs):
        model.train()
        ep_loss = 0.
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            ep_loss += loss.item()
        train_losses.append(ep_loss / len(train_loader))

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), torch.tensor(labels).to(device)
                correct += (model(imgs).argmax(1) == labels).sum().item()
                total   += len(labels)
        val_accs.append(correct / total)

        if verbose and (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1:3d}/{n_epochs} | Loss: {train_losses[-1]:.4f} | '
                  f'Val Acc: {val_accs[-1]:.4f}')
    model.cpu()
    return train_losses, val_accs


def extract_backbone_features(model, loader):
    """fc 이전 backbone으로 특징 추출 → (N, 512) tensor"""
    backbone = nn.Sequential(*list(model.children())[:-1])  # fc 제거
    backbone.eval().to(device)
    feats, labs = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            out = backbone(imgs.to(device)).squeeze(-1).squeeze(-1)  # (B, 512)
            feats.append(out.cpu())
            labs.append(torch.tensor(labels))
    backbone.cpu()
    return torch.cat(feats), torch.cat(labs)


# ── Feature Extraction 실험 ───────────────────────────────────────────────────
print('=== 실험 A: Random Init (전체 학습) ===')
torch.manual_seed(42)
model_rand = make_random_resnet18()
set_trainable(model_rand, None)   # 전체 학습
print(f'학습 파라미터: {count_trainable(model_rand):,} / 전체: {count_params(model_rand):,}')
rand_losses, rand_accs = train_model(model_rand, train_ldr, test_ldr, n_epochs=20)

print('\n=== 실험 B: Feature Extraction (backbone 고정, head만 학습) ===')
torch.manual_seed(42)
model_fe = make_pretrained_resnet18()
set_trainable(model_fe, ['fc'])   # fc만 학습
print(f'학습 파라미터: {count_trainable(model_fe):,} / 전체: {count_params(model_fe):,}')
fe_losses, fe_accs = train_model(model_fe, train_ldr, test_ldr, n_epochs=20)

# ── 비교 시각화 ──────────────────────────────────────────────────────────────
ep = range(1, 21)
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(ep, rand_losses, 'tomato',   lw=2, label='Random Init')
axes[0].plot(ep, fe_losses,   'seagreen', lw=2, label='Feature Extraction')
axes[0].set_title('Train Loss'); axes[0].set_xlabel('에포크')
axes[0].set_ylabel('CE Loss'); axes[0].legend()

axes[1].plot(ep, rand_accs, 'tomato',   lw=2, label=f'Random Init ({rand_accs[-1]:.4f})')
axes[1].plot(ep, fe_accs,   'seagreen', lw=2, label=f'Feature Extraction ({fe_accs[-1]:.4f})')
axes[1].axhline(y=1/3, color='gray', ls='--', lw=1, label='무작위 기준선 (0.333)')
axes[1].set_title('Val Accuracy'); axes[1].set_xlabel('에포크')
axes[1].set_ylabel('Accuracy'); axes[1].set_ylim(0, 1.05); axes[1].legend(fontsize=9)

plt.suptitle('Random Init vs Feature Extraction — 합성 도형 분류 (20 에포크)', fontsize=12)
plt.tight_layout(); plt.show()

print(f'\n최종 Val Accuracy:')
print(f'  Random Init       : {rand_accs[-1]:.4f}  ({count_trainable(model_rand):,} params 학습)')
print(f'  Feature Extraction: {fe_accs[-1]:.4f}  ({count_trainable(model_fe):,} params 학습)')
print(f'\n💡 Feature Extraction이 파라미터를 {count_trainable(model_rand)//count_trainable(model_fe)}배 적게 학습하면서도 성능이 좋습니다!')


---
## Part 2. Fine-tuning 전략

### 2-1. Fine-tuning이란?

Feature Extraction은 backbone을 완전히 고정합니다.  
**Fine-tuning**은 backbone까지 업데이트하여 새 도메인에 더 잘 맞게 조정합니다.

| 전략 | 학습 레이어 | 적합한 상황 |
|---|---|---|
| Feature Extraction | fc만 | 데이터 적음, 도메인 유사 |
| Partial Fine-tuning | layer4 + fc | 데이터 보통, 일부 적응 필요 |
| Full Fine-tuning | 전체 | 데이터 충분, 도메인 다름 |

### 2-2. Differential Learning Rate (차등 학습률)

```python
optimizer = optim.Adam([
    {'params': backbone_params, 'lr': 1e-4},   # backbone: 소폭 업데이트
    {'params': head_params,     'lr': 1e-3},   # head: 빠른 학습
])
```

**이유:** backbone은 ImageNet에서 이미 좋은 특징 학습 완료  
→ 큰 LR로 업데이트하면 **Catastrophic Forgetting** (기존 특징 손상) 위험

### 2-3. Catastrophic Forgetting

- 너무 큰 LR로 fine-tuning 시 → ImageNet 특징을 잊고 오히려 성능 하락
- 해결: 차등 LR, gradual unfreezing, 작은 LR


In [ ]:
# ── Fine-tuning (차등 LR) ─────────────────────────────────────────────────
print('=== 실험 C: Full Fine-tuning (차등 LR) ===')

torch.manual_seed(42)
model_ft = make_pretrained_resnet18()
set_trainable(model_ft, None)   # 전체 unfreeze
print(f'학습 파라미터: {count_trainable(model_ft):,}')

def diff_lr_optimizer(model, backbone_lr=1e-4, head_lr=1e-3):
    """backbone과 head에 차등 학습률 적용"""
    backbone_params = [p for n, p in model.named_parameters() if 'fc' not in n]
    head_params     = list(model.fc.parameters())
    return optim.Adam([
        {'params': backbone_params, 'lr': backbone_lr},
        {'params': head_params,     'lr': head_lr},
    ], weight_decay=1e-4)

ft_losses, ft_accs = train_model(
    model_ft, train_ldr, test_ldr, n_epochs=20,
    optimizer_fn=diff_lr_optimizer
)

print(f'\n최종 Val Accuracy (Fine-tuning): {ft_accs[-1]:.4f}')
print('→ backbone까지 새 도메인에 적응하므로 Feature Extraction보다 높거나 비슷합니다.')


In [ ]:
# ── 세 가지 전략 비교 시각화 ─────────────────────────────────────────────────

ep = range(1, 21)
strategies = [
    ('Random Init',          rand_losses, rand_accs, 'tomato',    count_trainable(model_rand)),
    ('Feature Extraction',   fe_losses,   fe_accs,   'steelblue', count_trainable(model_fe)),
    ('Fine-tuning (차등 LR)', ft_losses,   ft_accs,   'seagreen',  count_trainable(model_ft)),
]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for name, losses, accs, color, _ in strategies:
    axes[0].plot(ep, losses, color=color, lw=2, label=name)
    axes[1].plot(ep, accs,   color=color, lw=2, label=f'{name} ({accs[-1]:.4f})')

axes[0].set_title('Train Loss 비교'); axes[0].set_xlabel('에포크')
axes[0].set_ylabel('CE Loss'); axes[0].legend(fontsize=9)

axes[1].axhline(y=1/3, color='gray', ls='--', lw=1, label='무작위 기준선')
axes[1].set_title('Val Accuracy 비교'); axes[1].set_xlabel('에포크')
axes[1].set_ylabel('Accuracy'); axes[1].set_ylim(0, 1.05); axes[1].legend(fontsize=9)

plt.suptitle('Transfer Learning 전략 비교 — 합성 도형 분류 (20 에포크)', fontsize=12)
plt.tight_layout(); plt.show()

# 요약 표
print(f'\n{"전략":<25} {"Val Acc":>10} {"학습 파라미터":>15}')
print('-' * 54)
for name, _, accs, _, n_p in strategies:
    mark = ' ★' if accs[-1] == max(s[2][-1] for s in strategies) else ''
    print(f'{name:<25} {accs[-1]:>10.4f} {n_p:>15,}{mark}')

print('\n📌 핵심 관찰:')
print('  - Feature Extraction: 적은 파라미터로 놀라운 성능 → 소량 데이터에 최적')
print('  - Fine-tuning: backbone 적응 → 더 높은 잠재 성능, 충분한 데이터 필요')
print('  - Random Init: 사전지식 없이 시작 → 같은 데이터로 상대적 열위')


---
## Part 3. 데이터 효율성 + 오류 분석

### 3-1. 핵심 질문

> **"데이터가 얼마나 있어야 Transfer Learning이 의미 있을까?"**

- 일반적으로 **데이터가 적을수록** Transfer Learning의 이점이 크다
- 극단적 경우: 클래스당 10개 이미지만 있어도 사전학습 모델이 유리

### 3-2. Pre-computation 최적화

```python
# Backbone을 1회만 통과 → 512차원 특징 벡터 저장
feat_pre, y = extract_backbone_features(pretrained_model, train_ldr)
# → 이후 Linear Head를 특징 벡터로만 학습 (매우 빠름)
```

이 방식은 backbone 실행을 1회로 줄여 데이터 크기 실험을 빠르게 할 수 있습니다.


In [ ]:
# ── 데이터 효율성 분석 (Pre-computed Features 활용) ─────────────────────────

print('Backbone 특징 추출 중...')
t0 = time.time()

# 사전학습 backbone 특징 추출 (1회)
_pretrained_for_feat = make_pretrained_resnet18()
feat_pre_tr, y_tr_feat = extract_backbone_features(_pretrained_for_feat, train_ldr)
feat_pre_te, y_te_feat = extract_backbone_features(_pretrained_for_feat, test_ldr)

# 랜덤 초기화 backbone 특징 추출 (1회)
_random_for_feat = make_random_resnet18()
feat_rand_tr, _ = extract_backbone_features(_random_for_feat, train_ldr)
feat_rand_te, _ = extract_backbone_features(_random_for_feat, test_ldr)

print(f'특징 추출 완료: {time.time()-t0:.1f}s')
print(f'  Train features: {feat_pre_tr.shape}  Test features: {feat_pre_te.shape}')
del _pretrained_for_feat, _random_for_feat


def train_linear_head(feat_tr, y_tr, feat_te, y_te, n_epochs=50, lr=0.01):
    """Pre-computed features로 선형 분류기 학습 → Val Accuracy 반환"""
    model = nn.Linear(feat_tr.shape[1], len(CLASS_NAMES)).to(device)
    opt   = optim.Adam(model.parameters(), lr=lr)
    crit  = nn.CrossEntropyLoss()

    ds  = TensorDataset(feat_tr, y_tr)
    ldr = DataLoader(ds, batch_size=64, shuffle=True)

    for _ in range(n_epochs):
        model.train()
        for Xb, yb in ldr:
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            crit(model(Xb), yb).backward()
            opt.step()

    model.eval()
    with torch.no_grad():
        acc = (model(feat_te.to(device)).argmax(1) == y_te.to(device)).float().mean().item()
    model.cpu()
    return acc


# 데이터 크기별 비교 (클래스당 샘플 수)
n_sizes    = [30, 60, 100, 150, 200, 300]
accs_pre   = []
accs_rand  = []

print('\n데이터 효율성 실험 중...')
print(f'{"n/class":>8} {"Pretrained":>12} {"Random Init":>12}')
print('-' * 36)

for n in n_sizes:
    n_total = n * len(CLASS_NAMES)
    # n개/클래스를 순서대로 추출 (각 클래스 300개가 순서대로 있음)
    idx = []
    for cls in range(len(CLASS_NAMES)):
        idx.extend(range(cls * N_TRAIN_PER, cls * N_TRAIN_PER + n))
    idx = torch.tensor(idx)

    acc_p = train_linear_head(feat_pre_tr[idx],  y_tr_feat[idx], feat_pre_te,  y_te_feat)
    acc_r = train_linear_head(feat_rand_tr[idx], y_tr_feat[idx], feat_rand_te, y_te_feat)
    accs_pre.append(acc_p)
    accs_rand.append(acc_r)
    print(f'{n:>8} {acc_p:>12.4f} {acc_r:>12.4f}')

# 시각화
fig, ax = plt.subplots(figsize=(9, 5))
n_total_list = [n * len(CLASS_NAMES) for n in n_sizes]
ax.plot(n_total_list, accs_pre,  'seagreen', lw=2.5, marker='o', ms=8,
        label='Pretrained backbone (ImageNet)')
ax.plot(n_total_list, accs_rand, 'tomato',   lw=2.5, marker='s', ms=8,
        label='Random Init backbone')
ax.axhline(y=1/3, color='gray', ls='--', lw=1, label='무작위 기준선 (1/3)')
ax.fill_between(n_total_list,
                [max(0, p-r) for p,r in zip(accs_pre, accs_rand)],
                [0]*len(n_sizes),
                alpha=0.15, color='seagreen', label='Transfer Learning 이득')
ax.set_title('데이터 효율성: Transfer Learning vs 랜덤 초기화\n'
             '(Pre-computed backbone features → Linear Classifier)', fontsize=11)
ax.set_xlabel('학습 샘플 수 (전체)'); ax.set_ylabel('Test Accuracy')
ax.set_ylim(0, 1.05); ax.legend(fontsize=10)
plt.tight_layout(); plt.show()

print('\n📌 핵심:')
print('  - 소량 데이터(n=30/class, 90개)에서도 Pretrained >> Random')
print('  - 데이터가 늘어날수록 둘의 격차가 줄어드는 경향')
print('  - 실제 산업 현장: 라벨링 비용 때문에 소량 데이터가 일반적 → TL 필수')


In [ ]:
# ── 오류 분석: 혼동 행렬 + 잘못 분류된 샘플 ────────────────────────────────
from sklearn.metrics import confusion_matrix

# Fine-tuning 모델로 테스트셋 전체 예측
model_ft.eval()
all_preds, all_labels, all_imgs_raw = [], [], []

with torch.no_grad():
    for imgs, labels in test_ldr:
        preds = model_ft(imgs).argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(torch.tensor(labels).numpy())

all_preds  = np.array(all_preds)
all_labels = np.array(all_labels)
final_acc  = (all_preds == all_labels).mean()

# 혼동 행렬
cm = confusion_matrix(all_labels, all_preds)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES, annot_kws={'size': 14})
axes[0].set_title(f'혼동 행렬 (Fine-tuning, Test Acc={final_acc:.4f})', fontsize=10)
axes[0].set_xlabel('예측 (Predicted)'); axes[0].set_ylabel('실제 (True)')

# 잘못 분류된 샘플 시각화
wrong_idx = np.where(all_preds != all_labels)[0]
n_show    = min(6, len(wrong_idx))

if n_show > 0:
    selected = wrong_idx[:n_show]
    for i, idx in enumerate(selected):
        ax = axes[1] if n_show == 1 else plt.subplot(2, 6, i+7)
        pass

# 별도 figure로 오류 샘플 시각화
axes[1].axis('off')
axes[1].set_title('잘못 분류된 샘플 (아래 별도 그래프 참조)', fontsize=10)

plt.suptitle(f'Fine-tuning 모델 오류 분석 (Test Accuracy: {final_acc:.4f})', fontsize=12)
plt.tight_layout(); plt.show()

# 오류 샘플 시각화 (원본 PIL 이미지)
if len(wrong_idx) > 0:
    n_show = min(8, len(wrong_idx))
    fig, axes = plt.subplots(1, n_show, figsize=(3 * n_show, 3.5))
    if n_show == 1: axes = [axes]

    for i, idx in enumerate(wrong_idx[:n_show]):
        orig_img = test_ds.imgs[idx]   # 원본 PIL 이미지
        axes[i].imshow(orig_img)
        axes[i].set_title(
            f'실제: {CLASS_NAMES[all_labels[idx]]}\n예측: {CLASS_NAMES[all_preds[idx]]}',
            fontsize=9, color='red'
        )
        axes[i].axis('off')

    plt.suptitle(f'잘못 분류된 샘플 ({len(wrong_idx)}개 중 {n_show}개 표시)', fontsize=11)
    plt.tight_layout(); plt.show()
else:
    print('모든 테스트 샘플 정확 분류!')

print(f'\n오류 샘플 수: {len(wrong_idx)} / {len(all_labels)} ({len(wrong_idx)/len(all_labels)*100:.1f}%)')
print('\n클래스별 정확도:')
for cls in range(len(CLASS_NAMES)):
    mask = all_labels == cls
    acc_cls = (all_preds[mask] == all_labels[mask]).mean()
    print(f'  {CLASS_NAMES[cls]:<12}: {acc_cls:.4f}')


---
## Exercise

### Exercise 1. Fine-tuning Depth 탐색

**레이어 언프리즈 깊이**에 따른 성능을 비교하세요.

| 설정 | 학습 레이어 | 예상 학습 파라미터 |
|---|---|---|
| `head only` | fc | ~1,539 |
| `layer4 + head` | layer4, fc | ~2.6M |
| `layer3+4 + head` | layer3, layer4, fc | ~7.1M |
| `all` | 전체 | 11.7M |

**요구사항:**
- 동일 조건: `n_epochs=15, lr=3e-4, weight_decay=1e-4`
- `set_trainable()` 함수 활용
- 각 설정의 학습 파라미터 수 + 최종 Val Accuracy 표 출력
- 막대그래프 시각화 (x=설정, y=Val Acc)


In [ ]:
# Exercise 1: Fine-tuning Depth 탐색

configs = {
    'head only':        ['fc'],
    'layer4 + head':    ['layer4', 'fc'],
    'layer3+4 + head':  ['layer3', 'layer4', 'fc'],
    'all layers':       None,   # None = 전체 unfreeze
}
ex1_results = {}   # {config_name: (val_acc, n_trainable)}

for name, layers in configs.items():
    print(f'--- {name} 학습 중 ---')
    torch.manual_seed(42)
    # Your code here:
    # 1) make_pretrained_resnet18() 생성
    # 2) set_trainable(m, layers) 적용
    # 3) print 학습 파라미터 수
    # 4) train_model() 호출 (n_epochs=15, lr=3e-4)
    # 5) ex1_results[name] = (accs[-1], count_trainable(m))
    pass

# Your code here: 결과 표 출력
# print(f'{"설정":<20} {"Val Acc":>10} {"학습 파라미터":>15}')
# ...

# Your code here: 막대그래프 시각화 (x=설정명, y=Val Acc)


### Exercise 2. Data Augmentation 효과

Augmentation 유무에 따른 Fine-tuning 성능을 비교하세요.

**`aug_transform`** (이미 setup에서 정의됨):
```python
aug_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.ToTensor(), transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])
```

**요구사항:**
1. `ShapeDataset(N_TRAIN_PER, split='train', transform=aug_transform)` 생성
2. No Aug vs Aug: Full Fine-tuning (n_epochs=20) Val Accuracy 학습 곡선 비교
3. 관찰: Augmentation이 성능에 어떤 영향을 미치는가?


In [ ]:
# Exercise 2: Data Augmentation 효과

# Your code here: aug_transform 적용 DataLoader 생성
# train_ds_aug = ShapeDataset(N_TRAIN_PER, split='train', transform=aug_transform)
# train_ldr_aug = DataLoader(train_ds_aug, batch_size=32, shuffle=True, num_workers=0)

# Your code here: No Aug Fine-tuning (ft_losses, ft_accs는 이미 Part 2에서 학습됨 → 재사용 가능)
# torch.manual_seed(42)
# model_aug = make_pretrained_resnet18()
# set_trainable(model_aug, None)
# _, accs_aug = train_model(model_aug, train_ldr_aug, test_ldr, n_epochs=20,
#                           optimizer_fn=diff_lr_optimizer)

# Your code here: 학습 곡선 비교 (No Aug vs Aug)
# 힌트: ft_accs (No Aug, Part2에서 학습) vs accs_aug (Aug)


### Exercise 3. (도전) t-SNE Feature Space 시각화

Random backbone과 Pretrained backbone의 특징 공간을 **t-SNE**로 2D 시각화하여
클래스 분리도를 비교하세요.

**요구사항:**
1. `feat_pre_te` (사전학습 특징)와 `feat_rand_te` (랜덤 특징) 사용 (Part3에서 추출됨)
2. `TSNE(n_components=2, random_state=42, perplexity=30)` 적용
3. 2열 subplot: Random backbone t-SNE | Pretrained backbone t-SNE
4. 각 클래스를 다른 색상으로 scatter plot
5. 관찰: 어느 backbone이 클래스를 더 잘 분리하는가?

**힌트:**
```python
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
emb  = tsne.fit_transform(feat.numpy())   # (N, 2)
```


In [ ]:
# Exercise 3 (도전): t-SNE Feature Space 시각화
from sklearn.manifold import TSNE

# feat_pre_te, feat_rand_te, y_te_feat 는 Part 3에서 추출됨
print(f'테스트 특징 shape: {feat_pre_te.shape}  (N=300, dim=512)')

# Your code here:
# 1) TSNE로 feat_rand_te, feat_pre_te 각각 2D 임베딩
# 2) 2열 subplot: 왼쪽=Random, 오른쪽=Pretrained
# 3) scatter plot (클래스별 색상: CLASS_NAMES 3가지)
# 4) 범례 추가
# 5) 관찰 결과 주석으로 작성

# 힌트:
# colors = ['tomato', 'steelblue', 'seagreen']
# for cls in range(3):
#     mask = y_te_feat.numpy() == cls
#     ax.scatter(emb[mask, 0], emb[mask, 1], c=colors[cls], label=CLASS_NAMES[cls], alpha=0.7)


---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **Transfer Learning** | ImageNet 사전학습 특징 재사용 → 소량 데이터에서 높은 성능 |
| **Feature Extraction** | backbone 고정, head만 교체·학습 → 빠르고 overfitting 적음 |
| **Fine-tuning** | backbone까지 새 도메인에 맞게 업데이트 → 높은 성능, 더 많은 데이터 필요 |
| **Differential LR** | backbone 0.1×LR, head 1×LR → 기존 특징 보존 + head 빠른 학습 |
| **Catastrophic Forgetting** | 큰 LR fine-tuning → ImageNet 특징 손상 위험; 차등 LR로 방지 |
| **Pre-computation** | backbone 특징을 1회만 추출 → linear head 학습이 밀리초 단위로 빠름 |
| **ResNet18 구조** | conv1→layer1~4→avgpool→fc; BasicBlock (2 conv + skip connection) |
| **ImageNet 정규화** | mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]; 사전학습 입력 맞춤 필수 |
| **Frozen Layers** | `param.requires_grad=False`; Adam이 해당 파라미터를 업데이트하지 않음 |
| **데이터 효율성** | 클래스당 30개만으로도 Pretrained >> Random; 실제 산업 현장의 핵심 이유 |

---

**다음 강의:** BERT, GPT — Transformer 기반 대규모 사전학습 + 파인튜닝 패러다임
